# Exercise 04

In [1]:
import pandas as pd
import numpy as np
import requests

## 1. Прочитайте файл JSON, который вы сохранили в ex02 

In [2]:
df = pd.read_json('../data/auto.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CarNumber  725 non-null    object 
 1   Refund     725 non-null    int64  
 2   Fines      725 non-null    float64
 3   Make       725 non-null    object 
 4   Model      716 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 28.4+ KB


- Один из столбцов имеет тип float , поэтому давайте определим его формат в pandas с помощью pd.options.display.float_format: значения с плавающей точкой должны отображаться с двумя десятичными знаками;
- В Model отсутствуют значения, ничего с ними не делайте; 

In [3]:
pd.options.display.float_format = '{:.2f}'.format

## 2. Обогатите фрейм данных, используя образец из этого фрейма данных 

- Создайте выборку из 200 новых наблюдений с помощью random_state = 21;
    - В выборке не должно быть новых комбинаций номера автомобиля, марки и модели, поэтому весь набор данных будет соответствовать этим условиям;
    - Нет ограничений на Refund и Fines, вы можете выбрать любое значение из этих столбцов случайным образом и использовать его для обозначения любого номера автомобиля;
- Объедините образец с исходным фреймом данных в новый фрейм данных con_cat_rows; 

In [4]:
rng = np.random.default_rng(seed=21) 

count = 200

car_numbers = df['CarNumber'].values
makes = df['Make'].values
models = df['Model'].values 

new_data = {
    'CarNumber': np.random.choice(car_numbers, size=count),
    'Refund': np.random.choice([1, 2], size=count),
    'Fines': np.random.randint(1000, 7000, size=count),
    'Make': np.random.choice(makes, size=count),
    'Model': np.random.choice(models, size=count)
}

df_new = pd.DataFrame(new_data)
df_new

,CarNumber,Refund,Fines,Make,Model
0,Y687HM197RUS,2,5675,Skoda,Focus
1,K330T8197RUS,2,5041,Ford,Focus
2,Y316E877RUS,2,4319,Ford,Focus
3,9943OH154RUS,1,4586,Ford,None
4,8603T8154RUS,2,5740,Skoda,Focus
...,...,...,...,...,...
195,Y038TO197RUS,2,5056,Ford,Corolla
196,O83597197RUS,2,6518,Ford,Focus
197,98037H178RUS,1,2069,Ford,Focus
198,8049HT125RUS,2,5813,Ford,Focus


In [5]:
concat_rows = pd.concat([df, df_new])
concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
195,Y038TO197RUS,2,5056.00,Ford,Corolla
196,O83597197RUS,2,6518.00,Ford,Focus
197,98037H178RUS,1,2069.00,Ford,Focus
198,8049HT125RUS,2,5813.00,Ford,Focus


## 3. Oбогатите фрейм данных concat_rows новым столбцом с сгенерированными данными

- Создайте серию с названием Year, используя случайные целые числа с 1980 по 2019 год; 
- Используйте np.random.seed(21) перед генерацией лет; 
- Объедините серию с фреймом данных и назовите ее fines;

In [6]:
rng = np.random.default_rng(seed=21)
years = pd.Series(np.random.randint(1980, 2019, size=len(concat_rows)), name='Year')
fines = pd.concat([concat_rows.reset_index(drop=True), years.reset_index(drop=True)], axis=1)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,2008
1,E432XX77RUS,1,6500.00,Toyota,Camry,1980
2,7184TT36RUS,1,2100.00,Ford,Focus,1995
3,X582HE161RUS,2,2000.00,Ford,Focus,2003
4,92918M178RUS,1,5700.00,Ford,Focus,2003
...,...,...,...,...,...,...
920,Y038TO197RUS,2,5056.00,Ford,Corolla,2000
921,O83597197RUS,2,6518.00,Ford,Focus,1989
922,98037H178RUS,1,2069.00,Ford,Focus,1997
923,8049HT125RUS,2,5813.00,Ford,Focus,2014


In [7]:
fines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CarNumber  925 non-null    object 
 1   Refund     925 non-null    int64  
 2   Fines      925 non-null    float64
 3   Make       925 non-null    object 
 4   Model      911 non-null    object 
 5   Year       925 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 43.5+ KB


## 4. Обогатите фрейм данных данными из другого фрейма данных

- Создайте новый фрейм данных с номерами автомобилей и их владельцами;
  - получите самые популярные фамилии ( вы можете найти файл  surname.json  во вложениях ) в США; 
  - создать новую серию с фамилиями (они не должны иметь cпециальных символов,таких как запятые, скобки и т.д.) на основе собранных вами данных, количество которых должно быть равно количеству уникальных автомобильных номеров, использованных в выборке (используйте random_state = 21);
  - создайте фреймданных владельцев с двумя столбцами: CarNumber и SURNAME;

In [8]:
top_surname = pd.read_json('../../datasets/surname.json')
top_surname

,0,1,2
0,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
...,...,...,...
96,WILLIAMS,1625252,3
97,WILSON,801882,14
98,WOOD,250715,84
99,WRIGHT,458980,35


In [9]:
top_surname.columns = top_surname.iloc[0]
top_surname = top_surname[1:]
top_surname.reset_index(drop=True, inplace=True)
top_surname

,NAME,COUNT,RANK
0,ADAMS,427865,42
1,ALLEN,482607,33
2,ALVAREZ,233983,92
3,ANDERSON,784404,15
4,BAILEY,277845,72
...,...,...,...
95,WILLIAMS,1625252,3
96,WILSON,801882,14
97,WOOD,250715,84
98,WRIGHT,458980,35


In [10]:
rng = np.random.default_rng(seed=21)
surname = pd.Series(np.random.choice(top_surname['NAME'], size=fines['CarNumber'].nunique()), name='Surname')
car_num_unique = fines['CarNumber'].drop_duplicates()
owners = pd.concat([car_num_unique.reset_index(drop=True), surname.reset_index(drop=True)], axis=1)
owners

,CarNumber,Surname
0,Y163O8161RUS,ADAMS
1,E432XX77RUS,WILSON
2,7184TT36RUS,PHILLIPS
3,X582HE161RUS,ROSS
4,92918M178RUS,JOHNSON
...,...,...
526,O136HO197RUS,ADAMS
527,O22097197RUS,BENNETT
528,M0309X197RUS,JOHNSON
529,O673E8197RUS,EVANS


- Добавьте еще 5 наблюдений в фрейм данных о штрафах (придумайте свои собственные идеи CarNumber и т. д.)

In [11]:

new_data = {
    'CarNumber': ['R888RR28RUS', 'Q999QQ28RU', 'Z777ZZ28RUS', 'A444AA28RUS', 'Y555YY28RUS'],
    'Refund': [2, 1, 1, 2, 1],
    'Fines': [2345, 5556, 7777, 1000, 2000],
    'Make': ['Zeekr', 'Zeekr', 'Haval', 'Haval', 'Geely'],
    'Model': ['001', 'X', 'H9', 'F7', 'Coolray'],
    'Year': [2022, 2021, 2023, 2024, 2025]
}

df_new = pd.DataFrame(new_data)

fines = pd.concat([fines, df_new]).reset_index(drop=True)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,2008
1,E432XX77RUS,1,6500.00,Toyota,Camry,1980
2,7184TT36RUS,1,2100.00,Ford,Focus,1995
3,X582HE161RUS,2,2000.00,Ford,Focus,2003
4,92918M178RUS,1,5700.00,Ford,Focus,2003
...,...,...,...,...,...,...
925,R888RR28RUS,2,2345.00,Zeekr,001,2022
926,Q999QQ28RU,1,5556.00,Zeekr,X,2021
927,Z777ZZ28RUS,1,7777.00,Haval,H9,2023
928,A444AA28RUS,2,1000.00,Haval,F7,2024


- Удалите последние 20 наблюдений из фрейма данных от владельцев и добавьте 3 новых наблюдения (они не совпадают с теми, которые вы добавляете в фрейм данных о штрафах);

In [12]:
owners = owners.iloc[:-20]

In [13]:
new_data = {
    'CarNumber': ['W111WW28RUS', 'S555SS28RUS', 'G666GG28RUS'],
    'Surname': ['PIPA', 'BOBA', 'VIVA']
}
df_new = pd.DataFrame(new_data)
owners = pd.concat([owners, df_new]).reset_index(drop=True)
owners

,CarNumber,Surname
0,Y163O8161RUS,ADAMS
1,E432XX77RUS,WILSON
2,7184TT36RUS,PHILLIPS
3,X582HE161RUS,ROSS
4,92918M178RUS,JOHNSON
...,...,...
509,O50197197RUS,LOPEZ
510,7608EE777RUS,WILLIAMS
511,W111WW28RUS,PIPA
512,S555SS28RUS,BOBA


- Cоедините оба фрейма данных:
  - Новый фрейме данных должны быть только номера автомобилей, которые существуют в обоих фреймах данных; 
  - Новый фрейме данных должны быть все номера автомобилей, которые существуют в обоих фреймах данных; 
  - Новый фрейме данных должны быть только номера автомобилей из фрейма данных штрафов;
  - Новый фрейме данных должны быть только номера автомобилей из фрейма данных владельцев;

In [14]:
df_inner = pd.merge(fines, owners, how='inner')
df_inner

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,Y163O8161RUS,2,3200.00,Ford,Focus,2008,ADAMS
1,E432XX77RUS,1,6500.00,Toyota,Camry,1980,WILSON
2,7184TT36RUS,1,2100.00,Ford,Focus,1995,PHILLIPS
3,X582HE161RUS,2,2000.00,Ford,Focus,2003,ROSS
4,92918M178RUS,1,5700.00,Ford,Focus,2003,JOHNSON
...,...,...,...,...,...,...,...
895,Y038TO197RUS,2,5056.00,Ford,Corolla,2000,JONES
896,O83597197RUS,2,6518.00,Ford,Focus,1989,DIAZ
897,98037H178RUS,1,2069.00,Ford,Focus,1997,CASTILLO
898,8049HT125RUS,2,5813.00,Ford,Focus,2014,STEWART


In [15]:

df_outer = pd.merge(fines, owners, how='outer')
df_outer

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,704687163RUS,2.00,1400.00,Ford,Focus,2009.00,COOK
1,704687163RUS,2.00,6655.00,Ford,Focus,2002.00,COOK
2,704787163RUS,2.00,2800.00,Ford,Focus,1993.00,MORALES
3,704987163RUS,2.00,8594.59,Ford,Focus,2015.00,KELLY
4,705287163RUS,2.00,2000.00,Ford,Focus,1986.00,JAMES
...,...,...,...,...,...,...,...
928,Y969O8197RUS,2.00,6248.00,Ford,Focus,1983.00,ROBERTS
929,Y973O8197RUS,2.00,8594.59,Ford,Focus,2012.00,HILL
930,Y973O8197RUS,1.00,34800.00,Ford,Focus,1983.00,HILL
931,Y973O8197RUS,1.00,69600.00,Ford,Focus,1991.00,HILL


In [16]:
df_left = pd.merge(fines, owners, how='left')
df_left

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,Y163O8161RUS,2,3200.00,Ford,Focus,2008,ADAMS
1,E432XX77RUS,1,6500.00,Toyota,Camry,1980,WILSON
2,7184TT36RUS,1,2100.00,Ford,Focus,1995,PHILLIPS
3,X582HE161RUS,2,2000.00,Ford,Focus,2003,ROSS
4,92918M178RUS,1,5700.00,Ford,Focus,2003,JOHNSON
...,...,...,...,...,...,...,...
925,R888RR28RUS,2,2345.00,Zeekr,001,2022,NaN
926,Q999QQ28RU,1,5556.00,Zeekr,X,2021,NaN
927,Z777ZZ28RUS,1,7777.00,Haval,H9,2023,NaN
928,A444AA28RUS,2,1000.00,Haval,F7,2024,NaN


In [17]:
df_right = pd.merge(fines, owners, how='right')
df_right

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,2008.00,ADAMS
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,2002.00,ADAMS
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1980.00,WILSON
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,1983.00,WILSON
4,E432XX77RUS,1.00,2822.00,Ford,Focus,2018.00,WILSON
...,...,...,...,...,...,...,...
898,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2018.00,WILLIAMS
899,7608EE777RUS,1.00,6025.00,Ford,Focus,2012.00,WILLIAMS
900,W111WW28RUS,NaN,NaN,NaN,NaN,NaN,PIPA
901,S555SS28RUS,NaN,NaN,NaN,NaN,NaN,BOBA


## 5. Cоздайте сводную таблицу из фрейма данных о штрафах, она должна выглядеть примерно так (значения - это суммы штрафов), но с учетом всех лет (значения могут отличаться для вас)

In [18]:
fines_pivot = fines.pivot_table(
    index=['Make', 'Model'],
    columns='Year',
    values='Fines',
    aggfunc='sum'
)

fines_pivot

Year                    1980      1981     1982      1983     1984     1985  \
Make       Model                                                              
Audi       Focus         NaN       NaN      NaN       NaN      NaN      NaN   
BMW        Focus         NaN       NaN      NaN       NaN      NaN      NaN   
Ford       Camry         NaN       NaN  3402.00       NaN      NaN      NaN   
           Corolla   1001.00       NaN      NaN       NaN      NaN      NaN   
           Focus   387016.00 170777.59 87030.17 130607.59 88217.00 49179.00   
           Golf          NaN       NaN      NaN       NaN      NaN      NaN   
           Jetta     5756.00       NaN      NaN       NaN      NaN      NaN   
           Mondeo        NaN       NaN      NaN       NaN      NaN      NaN   
           Octavia   6020.00       NaN      NaN   5480.00  3469.00      NaN   
           Passat        NaN       NaN      NaN       NaN  5539.00      NaN   
           Touareg       NaN       NaN      NaN       NaN      NaN      NaN   
Geely      Coolray       NaN       NaN      NaN       NaN      NaN      NaN   
Haval      F7            NaN       NaN      NaN       NaN      NaN      NaN   
           H9            NaN       NaN      NaN       NaN      NaN      NaN   
Skoda      Focus         NaN   4199.00  5740.00       NaN  6480.00      NaN   
           Octavia   2000.00   5600.00      NaN  37800.00 13000.00  3900.00   
Toyota     Camry     6500.00  13500.00      NaN  13000.00 16494.59  1000.00   
           Corolla       NaN       NaN      NaN    900.00 19534.00 12700.00   
           Focus         NaN       NaN      NaN   2188.00      NaN  2324.00   
           Golf          NaN       NaN      NaN       NaN      NaN      NaN   
           Passat        NaN       NaN      NaN       NaN      NaN      NaN   
Volkswagen Focus         NaN       NaN  2688.00       NaN      NaN      NaN   
           Golf       500.00   9300.00      NaN       NaN      NaN      NaN   
           Jetta         NaN       NaN      NaN       NaN      NaN      NaN   
           Passat        NaN   6800.00 30200.00       NaN      NaN      NaN   
           Touareg       NaN       NaN      NaN       NaN      NaN  6300.00   
Zeekr      001           NaN       NaN      NaN       NaN      NaN      NaN   
           X             NaN       NaN      NaN       NaN      NaN      NaN   

Year                    1986      1987     1988     1989  ...      2014  \
Make       Model                                          ...             
Audi       Focus         NaN       NaN      NaN      NaN  ...       NaN   
BMW        Focus         NaN       NaN      NaN      NaN  ...       NaN   
Ford       Camry         NaN       NaN      NaN      NaN  ...   2705.00   
           Corolla   2947.00       NaN      NaN      NaN  ...       NaN   
           Focus    43874.00 137741.00 99067.17 87351.76  ... 151182.59   
           Golf          NaN       NaN      NaN      NaN  ...       NaN   
           Jetta         NaN       NaN      NaN      NaN  ...       NaN   
           Mondeo        NaN       NaN      NaN      NaN  ...       NaN   
           Octavia       NaN       NaN      NaN      NaN  ...       NaN   
           Passat    6709.00       NaN      NaN      NaN  ...   1340.00   
           Touareg       NaN       NaN      NaN  3211.00  ...       NaN   
Geely      Coolray       NaN       NaN      NaN      NaN  ...       NaN   
Haval      F7            NaN       NaN      NaN      NaN  ...       NaN   
           H9            NaN       NaN      NaN      NaN  ...       NaN   
Skoda      Focus         NaN       NaN      NaN      NaN  ...       NaN   
           Octavia       NaN  22189.17      NaN  4829.00  ...       NaN   
Toyota     Camry         NaN       NaN      NaN 16094.59  ...       NaN   
           Corolla       NaN   7800.00  3200.00      NaN  ...       NaN   
           Focus     5611.00       NaN      NaN      NaN  ...       NaN   
           Golf          NaN       NaN      NaN      N

## 6. Сохранить данные fines и owners в файлы CSV без индекса

In [19]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)